# User prompt
The user define the research objective a prompt. I use openAI to identify relevant key words for the research.
Documentation:
* https://platform.openai.com/docs/guides/text?api-mode=responses&lang=python

In [59]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.responses import Response

# Load the API key
load_dotenv()

client: OpenAI = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Research objective
research_objective: str = """
At present, one of our clients is looking to speak with professionals who have insights about the emerging technologies 
in soft contact lens manufacturing, particularly non-injection moulded methods. They would broadly like to understand how 
these technologies are reshaping the industry—from on-demand manufacturing to smart, drug-delivery-enabled lenses.
"""

prompt: str = f"""
You are assisting a researcher in generating targeted search terms for academic and patent literature related to the research topic described below.

Return a JSON object with the following structure:
- "main_topic": a concise phrase (2–5 words) that reflects the core technological focus of the research
- "openalex": exactly 5 academic search terms, each a single word
- "patentview": exactly 5 patent-related keywords, each a single word
- "cpc_codes": exactly 5 valid CPC classification codes relevant to the topic

Instructions:
- Output must be valid JSON only — no markdown, comments, or extra text
- All terms in "openalex" and "patentview" must be single words (no spaces, no hyphens)
- Do NOT include any words or close variants from "main_topic" in "openalex" or "patentview"
- All terms across the fields must be unique — no repetition or synonyms
- Each term in "openalex" and "patentview" must be conceptually compatible with the "main_topic" so that combining them (e.g. "main_topic" AND "keyword") produces a realistic and meaningful research query
- Use language and terminology commonly found in scientific publications and patent documents

Research Objective:
\"\"\" 
{research_objective} 
\"\"\"
"""

# Selected model
# model="gpt-3.5-turbo",
# model="gpt-4-turbo",
# model="gpt-4o", 
GPT_MODEL = "gpt-4o-mini"

response: Response = client.responses.create(

    model=GPT_MODEL,
    input = prompt
)

# print(type(response))
print(response.output_text)

{
  "main_topic": "soft contact lenses",
  "openalex": ["manufacturing", "technology", "design", "materials", "optics"],
  "patentview": ["innovation", "delivery", "smart", "process", "application"],
  "cpc_codes": ["A61F2/16", "A61F2/20", "B29C45/00", "B29D11/00", "C08J5/18"]
}


In [66]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.responses import Response

# Load the API key
load_dotenv()

client: OpenAI = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Research objective
research_objective: str = """
At present, one of our clients is looking to speak with professionals who have insights about the emerging technologies 
in soft contact lens manufacturing, particularly non-injection moulded methods. They would broadly like to understand how 
these technologies are reshaping the industry—from on-demand manufacturing to smart, drug-delivery-enabled lenses.
"""

prompt: str = f"""
You are assisting a researcher in generating targeted search terms for academic and patent literature related to the research topic described below.

Return a JSON object with the following structure:
- "main_topic": a concise phrase (2–5 words) that reflects the core technological focus of the research
- "openalex": exactly 5 academic search terms, each a single word
- "patentview": exactly 5 patent-related keywords, each a single word
- "cpc_codes": exactly 5 valid CPC classification codes relevant to the topic

Instructions:
- Output must be valid JSON only — no markdown, comments, or extra text
- All terms in "openalex" and "patentview" must be single words (no spaces, no hyphens)
- Do NOT include any words or close variants from "main_topic" in "openalex" or "patentview"
- All terms across the fields must be unique — no repetition or synonyms
- Each term in "openalex" and "patentview" must be conceptually compatible with the "main_topic" so that combining them (e.g. "main_topic" AND "keyword") produces a realistic and meaningful research query
- Use language and terminology commonly found in scientific publications and patent documents

Research Objective:
\"\"\" 
{research_objective} 
\"\"\"
"""

# Selected model
GPT_MODEL = "gpt-4o-mini"

response: Response = client.responses.create(

    model=GPT_MODEL,
    input = prompt
)

# print(type(response))
print(response.output_text)

{
  "main_topic": "soft contact lenses",
  "openalex": [
    "manufacturing",
    "technology",
    "innovation",
    "materials",
    "design"
  ],
  "patentview": [
    "optics",
    "biocompatible",
    "hydrogels",
    "coatings",
    "sustainability"
  ],
  "cpc_codes": [
    "G02C7/04",
    "A61F2/34",
    "B29C45/15",
    "B29D99/00",
    "G01N33/53"
  ]
}


In [ ]:
import json

def load_json(response_output: Response) -> dict[str, str | list[str]]:
    # Load the json file into dictionary
    json_dict:dict[str, str | list[str]] = json.loads(response_output)
    # print("Parsed dict:", research_key_words)
    return json_dict

# Print the main topic
research_key_words = load_json(response.output_text)
print(research_key_words["main_topic"])
# print(research_key_words["openalex"][0])

soft contact lenses


# Openalex API
https://docs.openalex.org/

Valid parameters are: 
* apc_sum, 
* cited_by_count_sum, 
* cursor, 
* filter, 
* format, 
* group_by, 
* group-by, 
* group_bys, 
* group-bys, 
* mailto, 
* page, 
* per_page, 
* per-page, 
* q, 
* sample, 
* seed, 
* search, 
* select, 
* sort, 
* warm.'

In [ ]:
import requests
import pandas as pd
from typing import Any

def reconstruct_abstract(abstract_inverted_index: dict[str, list[int]]) -> str:
    '''
    Reconstruct the abstract from abstract_inverted_index
    '''
    
    # Some works don't have an abstract
    if not abstract_inverted_index:
        return ""
        
    # Variable to store the highest index
    max_value: int = 0
     # Loop through all the list of position in the abstract_inverted_index dictionary.
    for values in abstract_inverted_index.values():
        # Loop through all the index value
        for value in values:
            # identify the highest value index
            if value >= max_value:
                max_value = value
                
    # Create an empty list with abstract size        
    abstract: list[str] = [None] * (max_value +1)

    # Loop through each word in the abstract_inverted_index:
    for word, positions in abstract_inverted_index.items():
        # For each word, get the list of positions it appears in.
        for position in positions:
            # Insert each word into its correct position in the list.
            abstract[position]= word
                        
    # Join all the words in the list into a single string, separated by spaces.
    abstract_text: str = " ".join(abstract)
    # print("\n", abstract_text)
    
    return abstract_text

url: str = "https://api.openalex.org/works"

# TODO: create a loop to make the search on all the identified key words.
main_topic: str = research_key_words["main_topic"]
research_key_word: str = research_key_words["openalex"][0]

search_terms: str = f"({main_topic} AND {research_key_word})"
mailto: str = "adyl.elguamra@gmail.com" #For best performance, add your email to all API requests

per_page: int = 15 # By default there are 25 results per page
page: int = 1 # Get the result from page number

params: dict = {
    "search": search_terms, # searches across titles, abstracts, and fulltext.
    "per_page": per_page, 
    "page": page, # if needed I can loop over pages. for example from page 1 to 5 with a for loop
    "sort": "relevance_score:desc",
    "mailto": mailto   
}
response: requests.Response = requests.get(url, params=params)

if response.status_code == 200:
    
    data: dict[str, Any] = response.json()

    works: list[dict[str, Any]] = data.get("results", []) # access a key in dictionary

    # Extract info into list of dicts
    records: list = []
    # loop in the list
    for work in works:
        
        abstract_inverted_index: dict[str, list[int]] = work.get("abstract_inverted_index", [])
        # print(type(abstract_inverted_index))
        abstract = reconstruct_abstract(abstract_inverted_index)

        record: dict[str, Any] = {
            "title": work.get("title"),
            "abstract": abstract,
            "publication_date": work.get("publication_date"),
            "year": work.get("publication_year"),
            "citations": work.get("cited_by_count"),
            "authors": [auth["author"]["display_name"] for auth in work.get("authorships", [])],
            "openAlex id": work.get("id"),
        }
        records.append(record)

    print("Final URL:", response.url)
    print("JSON Data:", response.json())

    # Convert to DataFrame
    df: pd.DataFrame = pd.DataFrame(records)
    display(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

Final URL: https://api.openalex.org/works?search=%28soft+contact+lenses+AND+manufacturing%29&per_page=15&page=1&sort=relevance_score%3Adesc&mailto=adyl.elguamra%40gmail.com
JSON Data: {'meta': {'count': 26400, 'db_response_time_ms': 282, 'page': 1, 'per_page': 15, 'groups_count': None}, 'results': [{'id': 'https://openalex.org/W2044512869', 'doi': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'title': 'Biomedical applications of hydrogels: A review of patents and commercial products', 'display_name': 'Biomedical applications of hydrogels: A review of patents and commercial products', 'relevance_score': 639.4715, 'publication_year': 2014, 'publication_date': '2014-11-29', 'ids': {'openalex': 'https://openalex.org/W2044512869', 'doi': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'mag': '2044512869'}, 'language': 'en', 'primary_location': {'is_oa': True, 'landing_page_url': 'https://doi.org/10.1016/j.eurpolymj.2014.11.024', 'pdf_url': None, 'source': {'id': 'https://openalex.or

,title,abstract,publication_date,year,citations,authors,openAlex id
0,Biomedical applications of hydrogels: A review of patents and commercial products,"Hydrogels have become very popular due to their unique properties such as high water content, softness, flexibility and biocompatibility. Natural and synthetic hydrophilic polymers can be physically or chemically cross-linked in order to produce hydrogels. Their resemblance to living tissue opens up many opportunities for applications in biomedical areas. Currently, hydrogels are used for manufacturing contact lenses, hygiene products, tissue engineering scaffolds, drug delivery systems and wound dressings. This review provides an analysis of their main characteristics and biomedical applications. From Wichterle's pioneering work to the most recent hydrogel-based inventions and products on the market, it provides the reader with a detailed introduction to the topic and perspective on further potential developments.",2014-11-29,2014,2291,"[Enrica Caló, Vitaliy V. Khutoryanskiy]",https://openalex.org/W2044512869
1,Multistate Outbreak of Fusarium Keratitis Associated With Use of a Contact Lens Solution,"Fusarium keratitis is a serious corneal infection, most commonly associated with corneal injury. Beginning in March 2006, the Centers for Disease Control and Prevention received multiple reports of Fusarium keratitis among contact lens wearers.To define the specific activities, contact lens hygiene practices, or products associated with this outbreak.Epidemiological investigation of Fusarium keratitis occurring in the United States. A confirmed case was defined as keratitis with illness onset after June 1, 2005, with no history of recent ocular trauma and a corneal culture growing Fusarium species. Data were obtained by patient and ophthalmologist interviews for case patients and neighborhood-matched controls by trained personnel. Available Fusarium isolates from patients' clinical and environmental specimens were genotyped by multilocus sequence typing. Environmental sampling for Fusarium was conducted at a contact lens solution manufacturing plant.Keratitis infection with Fusarium species.As of June 30, 2006, we identified 164 confirmed case patients in 33 states and 1 US territory. Median age was 41 years (range, 12-83 years). Corneal transplantation was required or planned in 55 (34%). One hundred fifty-four (94%) of the confirmed case patients wore soft contact lenses. Forty-five case patients and 78 controls were included in the case-control study. Case patients were significantly more likely than controls to report using a specific contact lens solution, ReNu with MoistureLoc (69% vs 15%; odds ratio, 13.3; 95% confidence interval, 3.1-119.5). The prevalence of reported use of ReNu MultiPlus solution was similar between case patients and controls (18% vs 20%; odds ratio, 0.7; 95% confidence interval, 0.2-2.8). Fusarium was not recovered from the factory, warehouse, solution filtrate, or unopened solution bottles; production of implicated lots was not clustered in time. Among 39 isolates tested, at least 10 different Fusarium species were identified, comprising 19 unique multilocus genotypes.The findings from this investigation indicate that this outbreak of Fusarium keratitis was associated with use of ReNu with MoistureLoc contact lens solution. Contact lens users should not use ReNu with MoistureLoc.",2006-08-22,2006,592,"[Douglas C. Chang, Gavin B. Grant, Kerry O’Donnell, Kathleen Wannemuehler, Judith Noble‐Wang, Carol Y. Rao, Lara M. Jacobson, Claudia S. Crowell, Rodlescia Sneed, Felicia M.T. Lewis, Joshua K. Schaffzin, Marion Kainer, Carol A. Genese, Eduardo C. Alfonso, Dan B. Jones, Arjun Srinivasan, Scott K. Fridkin, Benjamin J. Park, for the Fusarium Keratitis Investigation Team]",https://openalex.org/W2116582646
2,Contact lens practice,Part 1 Introduction Historical perspective. The anterior eye Visual optics Clinical instruments Part 2 Soft contact lenses Soft lens

## Openalex: Scoring and justification  with openai

Documentation:
* https://platform.openai.com/docs/guides/embeddings

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import time

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding


print("Embedding query...")
query_embedding = get_embedding(research_objective)

df["text"] = df["title"] + ". " + df["abstract"]


print("Embedding papers...")
df["embedding"] = df["text"].apply(get_embedding)

df["similarity"] = df["embedding"].apply(
    lambda x: cosine_similarity([query_embedding], [x])[0][0]
)


def get_justification(title, abstract, objective):
    prompt = f"""
    You are the CEO, as well as a scientific and regulatory analyst, evaluating academic research for a company exploring new technologies in soft contact lenses.

    Below is a paper's title and abstract, followed by the company's research objective. Assess the paper across eight dimensions critical to industry adoption. Your evaluation should reflect both business and technical perspectives.

    Title:  
    {title}

    Abstract:  
    {abstract}

    Research Objective:  
    {objective}

    For each dimension below, start with **Yes** or **No**, followed by a **1–2 sentence explanation** based only on the title and abstract. Be **concise**, **specific**, and **fact-based**. Avoid speculation or vague generalizations.

    1. **Technical Relevance** – Does the core technology directly relate to soft contact lenses or relevant materials/devices?  
    2. **Innovation** – Is there a clear, specific novelty or improvement over existing technologies?  
    3. **Feasibility** – Is the approach practical for real-world use or scalable manufacturing?  
    4. **Regulatory Fit** – Does it show potential to meet medical device or material safety regulations?  
    5. **Commercial Potential** – Is there a clear path to productization, monetization, or licensing?  
    6. **Research Credibility** – Are the science, methods, or authors/institutions reputable?  
    7. **IP / Competition** – Is the idea likely protectable or positioned ahead of competitors?  
    8. **Overall Relevance** – Is the paper relevant overall? Start with Yes or No, then briefly explain why.

    Format your response exactly like this:

    Technical Relevance: Yes/No – [reason]  
    Innovation: Yes/No – [reason]  
    Feasibility: Yes/No – [reason]  
    Regulatory Fit: Yes/No – [reason]  
    Commercial Potential: Yes/No – [reason]  
    Research Credibility: Yes/No – [reason]  
    IP / Competition: Yes/No – [reason]  
    Overall Relevance: Yes/No – [reason]

    Only use what is stated or implied in the title and abstract.
    
    Return a valid JSON object with exactly the following keys:

    - "justification": a string with the full 8-dimension evaluation in the exact format described above
    - "is_relevant": a string, either "Yes" or "No", based on your answer to the "Overall Relevance" question

    Ensure the JSON object is valid and returned as plain text — no markdown or extra explanation.
    """


    try:
        response: Response = client.responses.create(
        model=GPT_MODEL,
        input = prompt
        )
        return response.output_text
    except Exception as e:
        return f"ERROR: {e}"
    
print("Generating GPT justifications...")

# justification_dict = load_json(get_justification(df.at[0,"title"], df.at[0,"abstract"], research_objective))
# justification_dict["is_relevant"]

# TODO: optimize the code with .apply()
# loop through the dataframe and update the paper justification
for idx, row in df.iterrows():
    justification_dict = justification_dict = load_json(get_justification(row["title"], row["abstract"], research_objective))
    df.at[idx, "justification"] = justification_dict["justification"]
    df.at[idx, "is_relevant"] = justification_dict["is_relevant"]
    

Embedding query...
Embedding papers...
Generating GPT justifications...


In [71]:
# Show full column contents and more columns
pd.set_option("display.max_colwidth", None)    # Show full text in cells
# pd.set_option("display.max_columns", None)     # Show all columns
# pd.set_option("display.width", 0)              # Auto-detect width (or set a high number)

df[["title","similarity", "is_relevant", "justification"]].head(15)

,title,similarity,is_relevant,justification
0,Biomedical applications of hydrogels: A review of patents and commercial products,0.769526,Yes,"Technical Relevance: Yes – The paper discusses hydrogels, which are used in manufacturing contact lenses. Innovation: No – The abstract does not indicate any specific novel advancements beyond existing uses of hydrogels. Feasibility: Yes – Hydrogels are established in the market and feasible for manufacturing processes. Regulatory Fit: Yes – The mention of biocompatibility suggests potential compliance with medical device regulations. Commercial Potential: Yes – The review identifies ongoing applications and therefore hints at productization opportunities. Research Credibility: Yes – The paper reviews existing patents and products, implying a credible overview of the field. IP / Competition: Yes – The discussion of patents suggests protectable innovations within hydrogels. Overall Relevance: Yes – The content of the paper is pertinent to soft contact lens technology and the research objective."
1,Multistate Outbreak of Fusarium Keratitis Associated With Use of a Contact Lens Solution,0.783250,No,"Technical Relevance: Yes – The paper addresses a specific incident related to contact lens solutions and their impact on soft contact lens users. Innovation: No – The paper does not introduce new technologies or improvements; it reports an outbreak connected to an existing product. Feasibility: No – The research focuses on outbreak investigation rather than manufacturing methods, making practical implementation unclear. Regulatory Fit: Yes – The findings indicate the requirement for safety evaluations in specific solutions, aligning with regulatory frameworks. Commercial Potential: No – The paper does not present a pathway for new product development or monetization, focusing instead on a negative outcome associated with a product. Research Credibility: Yes – The publication is based on extensive epidemiological research and includes data from health authorities like the CDC. IP / Competition: No – The study is more about addressing a health crisis than presenting a novel concept that would be protectable. Overall Relevance: No – While the paper is informative, its focus on an outbreak may not align with the exploration of new contact lens technologies as stated in the research objective."
2,Contact lens practice,0.820748,No,"Technical Relevance: Yes – The paper discusses various aspects of soft contact lenses, including materials and manufacturing processes. Innovation: No – There is no specific novelty or improvement mentioned in the abstract. Feasibility: Yes – The paper suggests established practices for manufacturing and fitting contact lenses. Regulatory Fit: Yes – Mention of clinical considerations implies compliance with medical device regulations is likely. Commercial Potential: No – The abstract does not indicate a clear path to commercialization or licensing opportunities. Research Credibility: Yes – The paper addresses well-known practices in the contact lens field, suggesting credible background knowledge. IP / Competition: No – There is no indication of unique ideas that might be protectable, nor does it discuss competitive positioning. Overall Relevance: No – While the paper covers soft contact lenses, it does not specifically align with the focus on emerging non-injection molded technologies."
3,Therapeutic Contact Lenses with Polymeric Vehicles for Ocular Drug Delivery: A Review,0.823800,Yes,"Technical Relevance: Yes – The technology discussed directly relates to soft contact lenses and ocular drug delivery. Innovation: Yes – The incorporation of novel polymeric vehicles represents an advancement over conventional ocular drug delivery methods. Feasibility: No – Challenges like drug leaching during storage and potential surface roughness issues raise questions about practical implementation. Regulatory Fit: Yes – The technology appears to align with medical device regulations, bu

## Openalex: trends in relevant papers